In [48]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pyepo
from pyepo.model.grb import optGrbModel
import torch
from torch import nn
from torch.utils.data import DataLoader
from gurobipy import Model, GRB, quicksum

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

#from sklearn_extra.cluster import KMedoids
import copy

In [49]:

# optimization model
class myModel(optGrbModel):
    def __init__(self, weights):
        self.weights = np.array(weights)
        self.num_item = len(weights[0])
        
        #Fixed parameters
        self.max_elec = 10
        self.max_wind = 10
        self.nominal_wind = 10
        self.min_production = 50    
        super().__init__()

    def _getModel(self):

        periods = 10

        initial_plan = Model("Gurobi.Optimizer")

# Definition of variables
        E_DW = initial_plan.addVars(periods, name="E_DW", lb=0)
        E_UP = initial_plan.addVars(periods, name="E_UP", lb=0)
        b = initial_plan.addVars(periods, vtype=GRB.BINARY, name="b")
        E_settle = initial_plan.addVars(periods, name="E_settle")
        qF = initial_plan.addVars(range(1, n_features + 2), name="qF")
        qH = initial_plan.addVars(range(1, n_features + 2), name="qH")
        hydrogen = initial_plan.addVars(periods, name="hydrogen", lb=0)
        forward_bid = initial_plan.addVars(periods, name="forward_bid")

# Objective: Maximize profit
        initial_plan.setObjective(
            quicksum(
                lambda_F[t] * forward_bid[t] +
                lambda_H * hydrogen[t] +
                lambda_DW[t] * E_DW[t] -
                lambda_UP[t] * E_UP[t]
                for t in periods
            ), GRB.MAXIMIZE
        )

        # Constraints
        # Max capacity
        for t in periods:
            initial_plan.addConstr(forward_bid[t] <= max_wind_capacity, name=f"wind_capacity_up_{t}")
            initial_plan.addConstr(forward_bid[t] >= -max_elec_capacity, name=f"wind_capacity_dw_{t}")
            initial_plan.addConstr(hydrogen[t] <= max_elec_capacity, name=f"elec_capacity_{t}")

        # Power surplus (POSITIVE), deficit (NEGATIVE)
        for t in periods:
            initial_plan.addConstr(E_real[t ] - forward_bid[t] - hydrogen[t] == E_settle[t], name=f"settlement_{t}")
        
        for t in periods:
            initial_plan.addConstr(E_settle[t] == E_DW[t] - E_UP[t], name=f"balancing_{t}")

        for day in days:
            initial_plan.addConstr(sum(hydrogen[t] for t in day) >= min_production, name=f"min_production_{days.index(day)}")

        # Finalizing model setup
        initial_plan.update()
        initial_plan.optimize()

        return initial_plan, initial_plan.getObjective().getValue()


# prediction model
class LinearRegression(nn.Module):

    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(num_feat, num_item)

    def forward(self, x):
        out = self.linear(x)
        return out

In [50]:
# load data
red = (0.77, 0, 0.05) # (196, 0, 13)
blue = (0.12, 0.24, 1) # (31, 61, 255)
# green = (0.31, 1, 0.34) # (79, 255, 87)
green = (0.122, 00.816, 0.51) # (31, 208, 130)
navyblue = (0, 0, 0.4) # (0, 0, 102)
black = (0, 0, 0)
white = (1, 1, 1)
cgreen = (0.57254902, 0.7254902 , 0.51372549) # (146, 185, 131)
cblue = (0.70196078, 0.83137255, 1) # (179, 212, 255)

top_domain = 53.32 # 90% quantile



def import_data(negative_prices=False):
    # import data and set constants
    all_data = pd.read_csv("2020_data.csv")
    prices_UP = np.maximum(all_data["UP"].to_numpy(),0)
    prices_DW = np.maximum(all_data["DW"].to_numpy(),0)
    prices_F = np.maximum(all_data["forward_RE"].to_numpy(),0)
    prices_forecast = np.maximum(all_data["forward_FC"].to_numpy(), 0)

    nominal_wind = 10
    features = all_data.loc[:, ["Offshore DK2", "Offshore DK1", "Onshore DK2", "Onshore DK1", "production_FC"]]
    features["forward"] = prices_F
    features_red = all_data.loc[:, ["production_FC"]]
    features_red["forward"] = prices_F
    realized = all_data.loc[:, "production_RE"].to_numpy()
    realized *= nominal_wind

    price_H = 35.199999999999996
    penalty = np.quantile(prices_UP, 0.95) # 95% quantile of deficit_settle price over all 2 years
    # penalty = 2 * price_H
    # penalty = np.max(prices_B) # Something HIGHER is needed apparently

    return (
        prices_UP,
        prices_DW,
        prices_F,
        prices_forecast,
        features,
        features_red,
        realized,
        price_H,
        penalty
    )

In [51]:
import os
# get current directory
path = os.getcwd()
import os.path

def getParent(path, levels = 1):
    common = path
 
    # Using for loop for getting
    # starting point required for
    # os.path.relpath()
    for i in range(levels + 1):
 
        # Starting point
        common = os.path.dirname(common)
 
    # Parent directory upto specified
    # level
    return os.path.relpath(path, common)


In [52]:
#Import data
(prices_UP,prices_DW,prices_F,prices_forecast,features,features_red,realized,price_H,penalty) = import_data()

periods = list(range(0, len(prices_F) )) # Total time considered 2020-2021
num_feat = 5 # size of feature
num_feat_rf = 2 # size of feature
num_item = 2 # number of predictons (Forward bid and Hydrogen)


